In [ ]:
!pip install -q langchain-google-genai chromadb pandas plotly ipywidgets langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import time
import getpass

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('real_estate_crm_data(1).csv')
print(f"Data loaded with {len(df)} rows and {len(df.columns)} columns.")

Saving real_estate_crm_data.csv to real_estate_crm_data (1).csv
Data loaded with 300 rows and 21 columns.


In [ ]:
class RAG_System:
    def __init__(self, dataframe):
        self.df = dataframe
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.2,
        )
        self.embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        self.vectorstore = None
        self.qa_chain = None
        self.db_path = "./chroma_db"

        self._rag_()

    def _rag_(self):
        documents = []
        for idx, row in self.df.iterrows():
            content = f"""
                      Client Info:
                      -Name {row['FullName']} ,(ID: {row['ClientID']})
                      -Contact: {row['Email']}, {row['Phone']}
                      -Location Prefrence : {row['PreferredCity']}
                      - Property Interest: {row['PropertyType']}, Size: {row['PropertySize']} sqft
                      - Budget Range: {row['BudgetRange']}
                      - Assigned Agent: {row['AssignedAgent']}
                      - Lead Status: {row['Status']}
                      - Lead Source: {row['LeadSource']}
                      - Engagement: {row['TotalInteractions']} interactions, {row['PropertyVisits']} visits
                      - Deal Value: ₹{row['DealValue']:,}
                      - Client Tags: {row['ClientTags']}
                      - Lead Date: {row['LeadDate']}
                      - Last Contact: {row['LastContact']}
                      """
            doc = Document(
                page_content=content.strip(),
                metadata={
                    'client_id': row['ClientID'],
                    'status': row['Status'],
                    'agent': row['AssignedAgent'],
                    'city': row['PreferredCity'],
                    'deal_value': row['DealValue']
                }
            )
            documents.append(doc)

        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory="./chroma_db"
        )

        prompt_template = """
        You are a real estate CRM analytics expert. Use the following context to answer questions about leads, clients, agents, and business performance.
        Context: {context}
        Question: {question}
        Provide a detailed, data-driven answer. If the question involves numbers or comparisons, be specific with figures.
        Answer:
        """
        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 6}),
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )

    def query(self, question):

     print("\n[INFO] Sending documents and query to the LLM. Please wait, this can take up to a minute...")
     result = self.qa_chain.invoke({"query": question})
     print("[INFO] Received response from the LLM.")
     return result['result'], result['source_documents']


In [ ]:
class CRMAnalytics:
  def __init__(self, dataframe):
      self.df = dataframe

  def revenue_by_agent(self):

      print("Revenue Performance by Agent")

      agent_stats = self.df.groupby('AssignedAgent').agg({
          'DealValue': ['sum', 'mean', 'count'],
          'Status': lambda x: (x == "Sold").sum(),
          'ClientID': 'count'
      }).round(2)

      agent_stats.columns = ['Total_Revenue', 'Avg_Deal_Value', 'Total_Deals', 'Conversions', 'Total_Leads']
      agent_stats['Conversion_Rate'] = (agent_stats['Conversions'] / agent_stats['Total_Leads'] * 100).round(1)
      agent_stats = agent_stats.sort_values('Total_Revenue', ascending=False)

      fig = px.bar(
          agent_stats.reset_index(), x='AssignedAgent', y='Total_Revenue',
          title='Revenue Perf by Agent',
          labels={'Total_Revenue': 'Total Revenue Rs.', 'AssignedAgent': 'Agent'},
          text="Total_Revenue"
      )
      fig.update_traces(texttemplate='₹%{text:,.0f}', textposition='outside')
      fig.update_layout(height=500)
      fig.show()

      print("Detailed Agent Performance:")
      display(agent_stats)
      return agent_stats

  def status_distribution_analysis(self):

        print( "Lead Status Distribution & Pipeline")
        status_stats = self.df.groupby('Status').agg({
            'ClientID': 'count',
            'DealValue': 'sum'
        }).rename(columns={'ClientID': 'Count', 'DealValue': 'Revenue'})

        # Pie chart making
        fig = px.pie(
            status_stats.reset_index(),
            values='Count',
            names='Status',
            title='Lead Status Distribution',
            hole=0.3
        )
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()

        # Pipeline chart making
        pipeline_value = status_stats[status_stats.index.isin(['Hot Lead', 'Prospect', 'Active'])]
        if not pipeline_value.empty:
            fig2 = px.bar(
                pipeline_value.reset_index(),
                x='Status',
                y='Revenue',
                title='Pipeline Value by Status',
                text='Revenue'
            )
            fig2.update_traces(texttemplate='₹%{text:,.0f}', textposition='outside')
            fig2.show()

        print(" Status Summary:")
        display(status_stats)
        return status_stats

In [42]:
def run_query_interface(rag_system,analytics):
  print("Real Estate RAG Query Interface")
  print("Type your questions and press enter. Type 'exit' to stop.")

  query = input("Your question: ").strip()
  if query.lower() == 'exit':
    print("Exiting Agent")
  q= query.lower()

  try:
    if "revenue" in q or "agent" in q:
      analytics.revenue_by_agent()
    elif "status" in q or "pipeline" in q or "distribution" in q:
      analytics.status_distribution_analysis()
    else:
      answer , sources = rag_system.query(query)
      print(answer)
      print("Based on the given documents and gemini 2.5-flash")
  except Exception as e:
    print(f"Error Processing the query:{e}")


rag_system_run = RAG_System(df)
analytics = CRMAnalytics(df)
run_query_interface(rag_system_run,analytics)


Real Estate RAG Query Interface
Type your questions and press enter. Type 'exit' to stop.
Your question: agent
Revenue Performance by Agent


Detailed Agent Performance:


,Total_Revenue,Avg_Deal_Value,Total_Deals,Conversions,Total_Leads,Conversion_Rate
AssignedAgent,,,,,,
Ankit Sharma,224037100,7001159.38,32,9,32,28.1
Shreya Jain,88348347,3046494.72,29,6,29,20.7
Priya Gupta,82545183,3301807.32,25,4,25,16.0
Amit Singh,77410718,2345779.33,33,7,33,21.2
Nisha Patel,72582327,2199464.45,33,4,33,12.1
Kavitha Reddy,48572689,1798988.48,27,3,27,11.1
Sunita Desai,48271842,1177362.00,41,6,41,14.6
Deepak Kumar,23372767,865658.04,27,2,27,7.4
Ravi Menon,21565130,770183.21,28,5,28,17.9
